<a href="https://colab.research.google.com/github/rayen03/Tumor-Detect/blob/main/Tumor-Detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install opencv-python matplotlib

In [ ]:
import tensorflow as tf
import os
import cv2
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataDir='/content/drive/MyDrive/Data'
train_folder ='/content/drive/MyDrive/Data/train'
valid_folder = '/content/drive/MyDrive/Data/valid'
test_folder = '/content/drive/MyDrive/Data/test'
os.listdir(dataDir)

['test', 'train', 'valid']

In [ ]:
dataDir='/content/drive/MyDrive/Data-copy'
train_folder ='/content/drive/MyDrive/Data-copy/train'
valid_folder = '/content/drive/MyDrive/Data-copy/valid'
test_folder = '/content/drive/MyDrive/Data-copy/test'
os.listdir(dataDir)

['test', 'train', 'valid']

In [ ]:
Cclasses = os.listdir(test_folder)
Cclasses

['adenocarcinoma', 'large.cell.carcinoma', 'normal', 'squamous.cell.carcinoma']

In [ ]:

#deleting corrupted files
from PIL import Image


def filter_images(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            file_path = os.path.join(root, filename)

            if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                try:

                    with Image.open(file_path) as img:
                        img.verify()
                except OSError:

                    print(f"Removing: {file_path}")
                    os.remove(file_path)
            else:

                print(f"Removing: {file_path} - Unsupported file type")
                os.remove(file_path)

filter_images(test_folder)
filter_images(train_folder)
filter_images(valid_folder)


FileNotFoundError: ignored

In [ ]:



# Function to count the number of files in a folder
def count_files(folder_path):
    try:
        with os.scandir(folder_path) as entries:
            return sum(1 for entry in entries if entry.is_file() and not entry.name.startswith('.'))
    except OSError as e:
        print(f"Error counting files in {folder_path}: {e}")
        return 0

# Function to calculate the percentage of files in a folder
def calculate_percentage(folder_path, total_files):
    folder_files = count_files(folder_path)
    return (folder_files / total_files) * 100 if total_files > 0 else 0

# Iterate through test, train, and valid folders
for folder in [test_folder, train_folder, valid_folder]:
    total_files = sum([count_files(os.path.join(folder, class_name)) for class_name in Cclasses])

    print(f"Folder: {folder}")
    for class_name in Cclasses:
        class_folder = os.path.join(folder, class_name)
        class_count = count_files(class_folder)
        class_percentage = calculate_percentage(class_folder, total_files)
        print(f"  Class: {class_name} - Count: {class_count} - Percentage: {class_percentage:.2f}%")

    print(f"  Total Files: {total_files}")
    print()



Folder: /content/drive/MyDrive/Data/test
  Class: adenocarcinoma - Count: 62 - Percentage: 30.24%
  Class: large.cell.carcinoma - Count: 40 - Percentage: 19.51%
  Class: normal - Count: 38 - Percentage: 18.54%
  Class: squamous.cell.carcinoma - Count: 65 - Percentage: 31.71%
  Total Files: 205

Folder: /content/drive/MyDrive/Data/train
  Class: adenocarcinoma - Count: 230 - Percentage: 33.92%
  Class: large.cell.carcinoma - Count: 114 - Percentage: 16.81%
  Class: normal - Count: 154 - Percentage: 22.71%
  Class: squamous.cell.carcinoma - Count: 180 - Percentage: 26.55%
  Total Files: 678

Folder: /content/drive/MyDrive/Data/valid
  Class: adenocarcinoma - Count: 23 - Percentage: 33.82%
  Class: large.cell.carcinoma - Count: 17 - Percentage: 25.00%
  Class: normal - Count: 13 - Percentage: 19.12%
  Class: squamous.cell.carcinoma - Count: 15 - Percentage: 22.06%
  Total Files: 68



In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

from keras.utils import to_categorical
from sklearn.utils import shuffle

#buiding generators
basic_gen = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    dtype = 'float32'
)
train_gen = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    rotation_range = 20,
    width_shift_range = 0.3,
    height_shift_range = 0.3,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True,
    dtype = 'float32'
)
#data gen
train_data = train_gen.flow_from_directory(
    train_folder,
    target_size = (224,224),
    batch_size = 16,
    class_mode = 'categorical'
)

valid_data = basic_gen.flow_from_directory(
    valid_folder,
    target_size = (224,224),
    batch_size = 16,
    class_mode = 'categorical'
)

test_data = basic_gen.flow_from_directory(
    test_folder,
    target_size = (224,224),
    batch_size = 16,
    class_mode = 'categorical',
    shuffle = False
)



Found 678 images belonging to 4 classes.
Found 68 images belonging to 4 classes.
Found 205 images belonging to 4 classes.


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.regularizers import l2
from keras.src.metrics.confusion_metrics import activations
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,MaxPooling2D,Flatten,BatchNormalization,Dropout,Activation

input_shape=(224,224,3)

base_model = VGG16(
    include_top = False,
    weights="imagenet",
    input_shape = input_shape
)

#freezing layer to use the model as feature extractor
for layer in base_model.layers:
    layer.trainable = False


model = Sequential()
model.add(base_model)
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(1024,activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(512,activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(256,activation = 'relu'))
model.add(Dense(128,activation = 'relu'))
model.add(Dense(4,activation = 'softmax'))
model.summary()
model.summary()
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 batch_normalization_5 (Bat  (None, 7, 7, 512)         2048      
 chNormalization)                                                
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 3, 3, 512)         0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 4608)              0         
                                                                 
 dense_5 (Dense)             (None, 1024)              4719616   
                                                                 
 dropout_2 (Dropout)         (None, 1024)             

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001
)
model.compile(
    optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy']
)
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5),
    tf.keras.callbacks.ModelCheckpoint('model.h5',save_best_only=True),
    tf.keras.callbacks.TensorBoard(log_dir='logs'),
    reduce_lr
]
results = model.fit(train_data, validation_data=valid_data, epochs=120, verbose=1)


Epoch 1/120
43/43 [==============================] - 18s 353ms/step - loss: 1.6806 - accuracy: 0.3938 - val_loss: 1.2378 - val_accuracy: 0.3824
Epoch 2/120
43/43 [==============================] - 15s 343ms/step - loss: 1.1867 - accuracy: 0.4587 - val_loss: 1.1577 - val_accuracy: 0.3676
Epoch 3/120
43/43 [==============================] - 15s 355ms/step - loss: 1.0713 - accuracy: 0.5029 - val_loss: 1.1766 - val_accuracy: 0.3824
Epoch 4/120
43/43 [==============================] - 15s 354ms/step - loss: 0.9818 - accuracy: 0.5221 - val_loss: 1.0206 - val_accuracy: 0.4853
Epoch 5/120
43/43 [==============================] - 15s 351ms/step - loss: 1.0018 - accuracy: 0.5560 - val_loss: 0.9896 - val_accuracy: 0.5000
Epoch 6/120
43/43 [==============================] - 15s 338ms/step - loss: 0.9983 - accuracy: 0.5324 - val_loss: 1.0348 - val_accuracy: 0.4853
Epoch 7/120
43/43 [==============================] - 15s 351ms/step - loss: 0.8685 - accuracy: 0.6121 - val_loss: 0.9840 - val_accuracy:

In [ ]:
loss, acc = model.evaluate(test_data,verbose = 1)

13/13 [==============================] - 46s 4s/step - loss: 0.3529 - accuracy: 0.8878


In [ ]:
model.save('/content/drive/MyDrive/ModelData')

In [ ]:
history = model.fit(train_data, validation_data=valid_data, epochs=50, callbacks=callbacks, initial_epoch=30, verbose=1)


Epoch 31/50
43/43 [==============================] - ETA: 0s - loss: 0.7578 - accuracy: 0.8673

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


43/43 [==============================] - 58s 1s/step - loss: 0.7578 - accuracy: 0.8673 - val_loss: 1.1495 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 32/50
43/43 [==============================] - 57s 1s/step - loss: 0.7335 - accuracy: 0.8835 - val_loss: 1.4585 - val_accuracy: 0.6471 - lr: 0.0010
Epoch 33/50
43/43 [==============================] - 59s 1s/step - loss: 0.8669 - accuracy: 0.8569 - val_loss: 1.4530 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 34/50
43/43 [==============================] - 59s 1s/step - loss: 0.8827 - accuracy: 0.8481 - val_loss: 0.9526 - val_accuracy: 0.8529 - lr: 0.0010
Epoch 35/50
43/43 [==============================] - 58s 1s/step - loss: 0.9886 - accuracy: 0.8333 - val_loss: 1.9121 - val_accuracy: 0.6618 - lr: 0.0010
Epoch 36/50
43/43 [==============================] - 59s 1s/step - loss: 1.0187 - accuracy: 0.8348 - val_loss: 2.7241 - val_accuracy: 0.6029 - lr: 0.0010
Epoch 37/50
43/43 [==============================] - 62s 1s/step - loss: 1.0508 - accura